In [ ]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.preprocessing import LabelEncoder

# Mapeamento dos graus
graus = ["estavel", "leve", "mediana", "alta"]

# Carregar os dados do JSON
with open('dados-proc-reborn.json', 'r') as f:
    dados = json.load(f)

# Corrigir inversão entre grau e variacao
for d in dados:
    # Corrige ictericia
    if isinstance(d["grau_ictericia"], (int, float)):
        d["variacao_ictericia"], d["grau_ictericia"] = d["grau_ictericia"], d["variacao_ictericia"]
    # Corrige cianose
    if isinstance(d["grau_cianose"], (int, float)):
        d["variacao_cianose"], d["grau_cianose"] = d["grau_cianose"], d["variacao_cianose"]

# Converter graus numéricos para nomes (caso estejam como 0, 1, 2, 3)
for d in dados:
    if isinstance(d["grau_ictericia"], int) or str(d["grau_ictericia"]).isdigit():
        d["grau_ictericia"] = graus[int(d["grau_ictericia"])]
    if isinstance(d["grau_cianose"], int) or str(d["grau_cianose"]).isdigit():
        d["grau_cianose"] = graus[int(d["grau_cianose"])]

# Salvar os dados corrigidos
with open('dados_corrigidos.json', 'w') as f:
    json.dump(dados, f, indent=4)

# Pré-processamento
X = [d["rgb_ant"] + d["rgb_atu"] for d in dados]
X = np.array(X, dtype=np.float32)

var_ictericia = np.array([d["variacao_ictericia"] for d in dados], dtype=np.float32)
var_cianose = np.array([d["variacao_cianose"] for d in dados], dtype=np.float32)

# Encoder para graus
label_encoder = LabelEncoder()
label_encoder.fit(graus)


# Modelo
entrada = Input(shape=(6,))
x = Dense(128, activation='relu')(entrada)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
x = Dense(16, activation='relu')(x)
out_var_ictericia = Dense(1, name='variacao_ictericia')(x)
out_var_cianose = Dense(1, name='variacao_cianose')(x)


model = Model(inputs=entrada, outputs=[
    out_var_ictericia,
    out_var_cianose
])

model.compile(
    optimizer='adam',
    loss={
        'variacao_ictericia': 'mse',
        'variacao_cianose': 'mse',
        },
    metrics={
        'variacao_ictericia': 'mse',
        'variacao_cianose': 'mse',
        }
)

model.fit(
    X,
    {
        'variacao_ictericia': var_ictericia,
        'variacao_cianose': var_cianose,
    },
    epochs=10, verbose=1
)

model.save("modelo_ictericia_cianose.keras")


Epoch 1/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - loss: 206.3349 - variacao_cianose_loss: 90.1206 - variacao_cianose_mse: 90.1206 - variacao_ictericia_loss: 116.2141 - variacao_ictericia_mse: 116.2141
Epoch 2/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - loss: 19.7165 - variacao_cianose_loss: 5.2865 - variacao_cianose_mse: 5.2865 - variacao_ictericia_loss: 14.4299 - variacao_ictericia_mse: 14.4299
Epoch 3/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - loss: 15.4088 - variacao_cianose_loss: 3.7482 - variacao_cianose_mse: 3.7482 - variacao_ictericia_loss: 11.6606 - variacao_ictericia_mse: 11.6606
Epoch 4/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - loss: 13.3962 - variacao_cianose_loss: 2.9200 - variacao_cianose_mse: 2.9200 - variacao_ictericia_loss: 10.4762 - variacao_ictericia_mse: 10.4762
Epoch 5/10
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 12.4428 - variacao_cianose_loss: 2.6042 - variacao_cianose_mse: 2.6042 - variacao_ictericia_loss: 9.8386 - variacao_icte

In [ ]:

def prever(rgb_ant, rgb_atu):
    entrada = np.array([rgb_ant + rgb_atu], dtype=np.float32)

    # Faz a predição
    saida_var_ictericia, \
    saida_var_cianose = model.predict(entrada, verbose=0)

    # ICETÍCIA
    var_ictericia = float(saida_var_ictericia[0][0])

    # CIANOSE
    var_cianose = float(saida_var_cianose[0][0])
    return {
        "variacao_ictericia": round(var_ictericia, 2),
        "variacao_cianose": round(var_cianose, 2)
    }

# Exemplo:
print(prever([160, 104, 50], [ 160, 104, 50]))

print(prever( [209, 162, 128], [223, 180, 158]))

print(prever( [209, 162, 128], [140, 118, 130]))

{'variacao_ictericia': 0.42, 'variacao_cianose': -0.51}
{'variacao_ictericia': -36.19, 'variacao_cianose': -0.15}
{'variacao_ictericia': -0.12, 'variacao_cianose': 113.89}
